# Energy consumption prediction using LSTM/GRU in PyTorch

## The ML Pipeline

In [1]:
# Built-in
import os
import time
import gc

# Third-party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Local module
from fct import GRUNet, LSTMNet, move_sliding_window

In [3]:
from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

In [4]:
print(torch.__version__)

2.7.1+cpu


# Evaluating models
#### __Note: Running the following codes needs at least 16GB of memory.__
Moving on to measuring the accuracy of both models, we'll now use our `evaluate()` function and test dataset.

## Load data

In [5]:
# Import of parameters
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [6]:
# Accès aux variables
label_col_index = config["label_col_index"]
inputs_cols_indices = config["inputs_cols_indices"]
window_size = config["window_size"]
num_files_for_dataset = config["num_files_for_dataset"]

In [7]:
# Paths
data_raw_path = Path.cwd().parent / "data" / "raw"
data_processed_path = Path.cwd().parent / "data" / "processed"
model_path = Path.cwd().parent / "models"

In [8]:
# The scaler objects will be stored in this dictionary so that our output test data from the model can be re-scaled during evaluation
label_scalers = {}

train_x = []
test_x = {}
test_y = {}

# Skipping the files we're not using
processing_files = [
    file for file in os.listdir(data_raw_path) if os.path.splitext(file)[1] == ".csv"
]

for file in tqdm_notebook(processing_files[:num_files_for_dataset]):
    print(f"Processing {file} ...")
    # Store csv file in a Pandas DataFrame
    df = pd.read_csv(os.path.join(data_raw_path, file), parse_dates=["Datetime"])

    # Processing the time data into suitable input formats
    df = df.assign(
        hour=df["Datetime"].dt.hour,
        dayofweek=df["Datetime"].dt.dayofweek,
        month=df["Datetime"].dt.month,
        dayofyear=df["Datetime"].dt.dayofyear,
    )
    df = df.sort_values("Datetime").drop("Datetime", axis=1)

    # Scaling the input data
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    data = sc.fit_transform(df.values)
    

    # Obtaining the scaler for the labels(usage data) so that output can be
    # re-scaled to actual value during evaluation
    label_sc.fit(df.iloc[:, label_col_index].values.reshape(-1, 1))
    label_scalers[file] = label_sc

    # Move the window
    inputs, labels = move_sliding_window(
        data,
        window_size,
        inputs_cols_indices=inputs_cols_indices,
        label_col_index=label_col_index,
    )
    
    # Redure the precision of data
    data = data.astype(np.float32)
    inputs = inputs.astype(np.float32)
    labels = labels.astype(np.float32)

    # CONCAT created instances from all .csv files.
    # Split data into train/test portions and combining all data from different files into a single array
    test_portion = int(0.1 * len(inputs))
    if len(train_x) == 0:  # first iteration
        train_x = inputs[:-test_portion]
        train_y = labels[:-test_portion]
    else:
        train_x = np.concatenate((train_x, inputs[:-test_portion]))
        train_y = np.concatenate((train_y, labels[:-test_portion]))
    test_x[file] = inputs[-test_portion:]
    test_y[file] = labels[-test_portion:]
    
    # Remove temporary variables
    del df, data, inputs, labels
    gc.collect()
    

  0%|          | 0/5 [00:00<?, ?it/s]

Processing AEP_hourly.csv ...
(121183, 90, 5) (121183, 1)
Processing COMED_hourly.csv ...
(66407, 90, 5) (66407, 1)
Processing DAYTON_hourly.csv ...
(121185, 90, 5) (121185, 1)
Processing DEOK_hourly.csv ...
(57649, 90, 5) (57649, 1)
Processing DOM_hourly.csv ...
(116099, 90, 5) (116099, 1)


## Load the GRU model

In [9]:
# Accès aux variables
n_hidden = config["n_hidden"]
hidden_dim = n_hidden
input_dim = config["input_dim"]
output_dim = config["output_dim"]
n_layers = config["n_layers"]

In [10]:
# move device to cpu for evaluation to avoid GPU memory run
device = "cpu"

In [11]:
gru_model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
gru_model.load_state_dict(torch.load(os.path.join(model_path, "gru_model.pt")))

<All keys matched successfully>

In [12]:
# Move the model to the appropriate device
gru_model.to(device)

GRUNet(
  (gru): GRU(5, 32, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
)

## Load the LSTM model

In [13]:
lstm_model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
lstm_model.load_state_dict(torch.load(os.path.join(model_path, "lstm_model.pt")))

<All keys matched successfully>

In [14]:
# Move the model to the appropriate device
lstm_model.to(device)

LSTMNet(
  (lstm): LSTM(5, 32, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
)

## Model Evaluation

For the purpose of comparing the performance of both models as well, we'll being tracking the time it takes for the model to train and eventually comparing the final accuracy of both models on the test set. For our accuracy measure, we'll use ***Symmetric Mean Absolute Percentage Error (sMAPE)*** to evaluate the models. *sMAPE* is the sum of the **absolute difference** between the predicted and actual values divided by the average of the predicted and actual value, therefore giving a percentage measuring the amount of error. 

This is the formula for *sMAPE*:

$sMAPE = \frac{100%}{n} \sum_{t=1}^n \frac{|F_t - A_t|}{(|F_t + A_t|)/2}$

In [15]:
def sMAPE(outputs, targets):
    sMAPE = (
        100
        / len(targets)
        * np.sum(np.abs(outputs - targets) / (np.abs(outputs + targets)) / 2)
    )
    return sMAPE

In [16]:
def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.process_time()
    # get data of test data for each state
    for file in test_x.keys():
        inputs = torch.from_numpy(np.array(test_x[file]))
        labels = torch.from_numpy(np.array(test_y[file]))

        h = model.init_hidden(inputs.shape[0])

        # predict outputs
        with torch.no_grad():
            out, h = model(inputs.to(device).float(), h)

        outputs.append(
            label_scalers[file]
            .inverse_transform(out.cpu().detach().numpy())
            .reshape(-1)
        )

        targets.append(
            label_scalers[file].inverse_transform(labels.numpy()).reshape(-1)
        )

    # Merge all files
    concatenated_outputs = np.concatenate(outputs)
    concatenated_targets = np.concatenate(targets)

    print(f"Evaluation Time: {time.process_time()-start_time}")
    print(f"sMAPE: {round(sMAPE(concatenated_outputs, concatenated_targets), 3)}%")

    # list of of targets/outputs for each state
    return outputs, targets, sMAPE

## Evaluate performance of GRU

In [17]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)

Evaluation Time: 19.500326863
sMAPE: 0.3529999852180481%


## Evaluate performance of LSTM

In [18]:
lstm_outputs, targets, lstm_sMAPE = evaluate(lstm_model, test_x, test_y, label_scalers)

Evaluation Time: 11.867153252999998
sMAPE: 0.5410000085830688%


In [29]:
targets

[array([16370., 16517., 16935., ..., 17001., 15964., 14809.],
       shape=(12118,), dtype=float32),
 array([11271.   , 11372.999, 11403.   , ..., 15086.   , 14447.999,
        13334.999], shape=(6640,), dtype=float32),
 array([2063., 2061., 2119., ..., 2405., 2250., 2042.],
       shape=(12118,), dtype=float32),
 array([3504.    , 3482.0002, 3380.    , ..., 3851.    , 3575.    ,
        3281.    ], shape=(5764,), dtype=float32),
 array([ 9618.   ,  9803.   ,  9870.   , ..., 13312.   , 12390.001,
        11385.   ], shape=(11609,), dtype=float32)]

While the GRU model may have made smaller errors and edged the LSTM model slightly in terms of performance accuracy, the difference is insignificant and thus inconclusive. There have been many other tests conducted by others comparing both these models but there has largely been no clear winner as to which is the better architecture overall.

In [30]:
len(
    gru_outputs
)  # list of predicted output file for each state (each element has a 1d array for that state)

5

In [21]:
gru_outputs

[array([16494.787, 16559.102, 16948.727, ..., 16809.125, 16386.562,
        14816.431], shape=(12118,), dtype=float32),
 array([11311.874, 11736.216, 11477.399, ..., 14871.391, 14466.329,
        13513.114], shape=(6640,), dtype=float32),
 array([2086.8677, 2086.4172, 2121.4314, ..., 2394.8381, 2298.2175,
        2074.6746], shape=(12118,), dtype=float32),
 array([3554.6465, 3468.8118, 3434.8777, ..., 3728.031 , 3737.2366,
        3251.2544], shape=(5764,), dtype=float32),
 array([ 9903.207, 10062.905, 10007.195, ..., 13397.379, 12485.42 ,
        11218.503], shape=(11609,), dtype=float32)]

# Some visualizations

Lastly, let's do some visualizations on random sets of our predicted output vs the actual consumption data for some states.

In [22]:
states_list = list(test_x.keys())

In [23]:
states_list

['AEP_hourly.csv',
 'COMED_hourly.csv',
 'DAYTON_hourly.csv',
 'DEOK_hourly.csv',
 'DOM_hourly.csv']

In [24]:
nb = 100
i = 0
df = pd.DataFrame({
    "Time Step": range(nb),  # ou range(-100, 0) si tu veux des indices négatifs
    "GRU Prediction": gru_outputs[i][-nb:],
    "LSTM Prediction": lstm_outputs[i][-nb:],
    "Actual": targets[i][-nb:]
})

In [25]:
df

,Time Step,GRU Prediction,LSTM Prediction,Actual
0,0,15613.262695,15596.736328,15616.000000
1,1,15185.230469,15045.432617,15476.000000
2,2,15036.260742,14420.304688,14665.000000
3,3,13748.802734,13593.167969,13680.000000
4,4,12943.632812,12940.184570,12927.000000
...,...,...,...,...
95,95,17792.246094,18040.521484,17673.001953
96,96,17262.681641,17280.298828,17303.000000
97,97,16809.125000,16539.115234,17001.000000
98,98,16386.562500,15874.004883,15964.000000


In [26]:
df.to_excel("df.xlsx", index=False)

In [27]:
df

,Time Step,GRU Prediction,LSTM Prediction,Actual
0,0,15613.262695,15596.736328,15616.000000
1,1,15185.230469,15045.432617,15476.000000
2,2,15036.260742,14420.304688,14665.000000
3,3,13748.802734,13593.167969,13680.000000
4,4,12943.632812,12940.184570,12927.000000
...,...,...,...,...
95,95,17792.246094,18040.521484,17673.001953
96,96,17262.681641,17280.298828,17303.000000
97,97,16809.125000,16539.115234,17001.000000
98,98,16386.562500,15874.004883,15964.000000


In [28]:
plt.figure(figsize=(14, 10))
plt.subplot(2, 2, 1)
plt.plot(gru_outputs[0][-100:], "-o", color="g", label="GRU Predictions", markersize=2)
plt.plot(
    lstm_outputs[0][-100:], "-o", color="r", label="LSTM Predictions", markersize=2
)
plt.plot(targets[0][-100:], color="b", label="Actual")
plt.ylabel("Energy Consumption (MW)")
plt.title(f"Energy Consumption for {states_list[0]} state")
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(gru_outputs[1][-50:], "-o", color="g", label="GRU Predictions", markersize=2)
plt.plot(lstm_outputs[1][-50:], "-o", color="r", label="LSTM Predictions", markersize=2)
plt.plot(targets[1][-50:], color="b", label="Actual")
plt.ylabel("Energy Consumption (MW)")
plt.title(f"Energy Consumption for {states_list[1]} state")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(gru_outputs[2][:50], "-o", color="g", label="GRU Predictions", markersize=2)
plt.plot(lstm_outputs[2][:50], "-o", color="r", label="LSTM Predictions", markersize=2)
plt.plot(targets[2][:50], color="b", label="Actual")
plt.ylabel("Energy Consumption (MW)")
plt.title(f"Energy Consumption for {states_list[2]} state")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(gru_outputs[3][:100], "-o", color="g", label="GRU Predictions", markersize=2)
plt.plot(lstm_outputs[3][:100], "-o", color="r", label="LSTM Predictions", markersize=2)
plt.plot(targets[3][:100], color="b", label="Actual")
plt.title(f"Energy Consumption for {states_list[3]} state")
plt.ylabel("Energy Consumption (MW)")
plt.legend()
plt.show()

AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

<Figure size 1400x1000 with 4 Axes>

Looks like the models are largely successful in predicting the trends of energy consumption. While they may still get some changes wrong, such as delays in predicting a drop in consumption, the predictions follow very closely to the actual line on the test set. This is due to the nature of energy consumption data and the fact that there are patterns and cyclical changes that the model can account for. Tougher time-series prediction problems such as stock price prediction or sales volume prediction may have data that is largely random or doesn’t have predictable patterns, and in such cases, the accuracy will definitely be lower.

## What's next?

* Use more data.
* Use more complex (more layers) networks.